# Integrated Gradient Correlation
## Example on a classification task with the MNIST dataset

### Init dataset and model

In [ ]:
import numpy as np
from mnist.model_mnist_1v0 import Dataset, Model

parameters = {
    'conv_sizes': (64, 128),
    'lin_sizes': (128, 64, 32, 16),
    'act_type': 'mish'}

dataset = Dataset()
model = Model(
    dataset, model_name='mnist_1v0_a000', trainable=True, device='cpu',
    parameters=parameters)

### Train model

In [ ]:
model.train(n_epoch=400)

### Compute model accuracy

In [ ]:
val_nll, val_ac = model.score()
print(np.round(val_nll, 3), np.round(val_ac, 5))

### Compute Integrated Gradient Correlation

In [ ]:
_ = model.int_grad_corr_val(x_0=32, n_steps=32, check_error=True)

### Visualize IGC maps

In [ ]:
from mnist.figures_1v0 import mnist_img

mean_digits = dataset.compute_mean_digits()
diff_probs = dataset.digit_diff_probabilities()
int_grad_corr = np.load(model.get_result_path('int_grad_corr.npy'))

fig = mnist_img(int_grad_corr, contour_m=mean_digits, contour_p=diff_probs)

### Plot summaries of IGC maps

In [ ]:
from mnist.figures_1v0 import mnist_plot

int_grad_corr = np.load(model.get_result_path('int_grad_corr.npy'))

mean_digits = dataset.compute_mean_digits()
mask_1 = mean_digits > np.quantile(
    mean_digits, 0.8, axis=(1, 2), keepdims=True)
inside = np.sum(int_grad_corr * mask_1, axis=(1, 2))
outside = np.sum(int_grad_corr * np.invert(mask_1), axis=(1, 2))

diff_probs = dataset.digit_diff_probabilities()
mask_2 = diff_probs > 0.5
high = np.sum(int_grad_corr * mask_2, axis=(1, 2))
low = np.sum(int_grad_corr * np.invert(mask_2), axis=(1, 2))

fig = mnist_plot(inside, outside, low, high)

### Visualize Integrated Gradients for few samples

In [ ]:
from mnist.figures_1v0 import mnist_img_10

x = dataset.get_10_digits(seed=103)
int_grad = np.zeros((
    dataset.n_digits, dataset.n_digits, dataset.img_size,
    dataset.img_size), dtype=np.float32)
for i in range(dataset.n_digits):
    int_grad[i] += model.int_grad_1_x(x[i], x_0=32, n_steps=32)[2]

fig = mnist_img_10(int_grad, contour_m=x)

In [ ]:
from mnist.figures_1v0 import mnist_img_10

digit = 4
x = dataset.get_10_digits(digit=digit, seed=103)
int_grad = np.zeros((
    dataset.n_digits, dataset.n_digits, dataset.img_size,
    dataset.img_size), dtype=np.float32)
for i in range(dataset.n_digits):
    int_grad[i] += model.int_grad_1_x(x[i], x_0=32, n_steps=32)[2]

fig = mnist_img_10(int_grad, contour_m=x, digit=digit)

### Compute Baseline Shapley Correlation

In [ ]:
_ = model.bsl_shap_corr_val(x_0=32, n_iter=32, check_error=True, n_samples=1000)

### Visualize BSC maps

In [ ]:
from mnist.figures_1v0 import mnist_img

mean_digits = dataset.compute_mean_digits()
diff_probs = dataset.digit_diff_probabilities()
bsl_shap_corr = np.load(model.get_result_path('bsl_shap_corr.npy'))

fig = mnist_img(bsl_shap_corr, contour_m=mean_digits, contour_p=diff_probs)

### Model without convolution layers

In [ ]:
import numpy as np
from mnist.model_mnist_1v0 import Dataset, Model

parameters = {
    'conv_sizes': None,
    'lin_sizes': (128, 64, 32, 16),
    'act_type': 'mish'}

dataset = Dataset()
model = Model(
    dataset, model_name='mnist_1v0_a100', trainable=True, device='cpu',
    parameters=parameters)

### Model with ReLU activation functions

In [ ]:
import numpy as np
from mnist.model_mnist_1v0 import Dataset, Model

parameters = {
    'conv_sizes': (64, 128),
    'lin_sizes': (128, 64, 32, 16),
    'act_type': 'relu'}

dataset = Dataset()
model = Model(
    dataset, model_name='mnist_1v0_a200', trainable=True, device='cpu',
    parameters=parameters)